In [16]:
import polars as pl

In [17]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [18]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is 5E9C-93AA

 Directory of C:\Users\mp5667di\Desktop\WSU___Fall__2024\DSCI_326\lab_1.2\Lab 1.2 - Processing a Healthcare Survey

09/10/2024  05:15 PM    <DIR>          .
09/10/2024  04:59 PM    <DIR>          ..
09/10/2024  08:25 AM                66 .gitattributes
09/10/2024  09:04 AM    <DIR>          .ipynb_checkpoints
09/10/2024  05:04 PM    <DIR>          data
09/10/2024  08:22 AM           402,064 lab1_2_processing_health_survey.docx
09/10/2024  05:15 PM            22,018 lab_1.2_healthcare_survey.ipynb
09/10/2024  08:25 AM                87 README.md
               4 File(s)        424,235 bytes
               4 Dir(s)  32,873,848,832 bytes free


In [19]:
health_survey = pl.read_csv('./data/health_survey.csv')
health_survey = (health_survey
                .with_row_index('Participant', offset=1)
                )
health_survey.head(2)

Participant,,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,F1.2,F2.2,F6.1,F2.3,F4.1,F2.4,F5.2,F2.5,F6.2,F1.3,F2.6,F5.3,F4.2,F2.7,F3.1,F2.8,F5.4,F3.2,F1.4,F3.3,F1.5,F5.5,F6.3,F1.6,F5.6,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
u32,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""


In [22]:
# Question 3-a 
# ***try using regular expression***

question_cols = ['F1','F5','F2','F1.1','F2.1','F6','F4','F3','F5.1','F1.2','F2.2','F6.1','F2.3','F4.1','F2.4','F5.2','F2.5','F6.2','F1.3','F2.6','F5.3',
                'F4.2','F2.7','F3.1','F2.8','F5.4','F3.2','F1.4','F3.3','F1.5','F5.5','F6.3','F1.6','F5.6','F2.9','F3.4','F4.3','F2.10','F1.7','F6.4',
                'F4.4','F5.7','F3.5','F2.11']

Q3_a = (health_survey
          .unpivot(question_cols,
                   index = 'Participant',
                   variable_name = 'Question',
                   value_name = 'Response')
          )
Q3_a.head()

Participant,Question,Response
u32,str,str
1,"""F1""","""Somewhat Agree"""
2,"""F1""","""Somewhat Agree"""
3,"""F1""","""Strongly Agree"""
4,"""F1""","""Somewhat Agree"""
5,"""F1""","""Strongly Agree"""


In [23]:
reverse_coding = pl.read_csv('./data/ReverseCodingItems.csv')
reverse_coding.head(2)

Question,Construct,Question # on Qualtrics Survey,Needs Reverse Coding?,Column Name
str,i64,i64,str,str
"""In the future, I plan to parti…",1,1,"""No""","""F1"""
"""Individuals are responsible fo…",5,2,"""Yes""","""F5"""


In [31]:
# Question 3-b

Q3_b = (Q3_a
        .join(reverse_coding,
             left_on = 'Question',
             right_on = 'Column Name',
             how = 'left')
        .rename({'Needs Reverse Coding?': 'NeedsReverse'})
        .drop(['Question_right','Construct','Question # on Qualtrics Survey'])
       )
Q3_b.head()

Participant,Question,Response,NeedsReverse
u32,str,str,str
1,"""F1""","""Somewhat Agree""","""No"""
2,"""F1""","""Somewhat Agree""","""No"""
3,"""F1""","""Strongly Agree""","""No"""
4,"""F1""","""Somewhat Agree""","""No"""
5,"""F1""","""Strongly Agree""","""No"""


In [59]:
# Question 3-c

Q3_c = (Q3_b
       .with_columns(TempRecodeValue = pl.when(pl.col('Response') == 'Strongly Disagree')
                                         .then(1)
                                         .when(pl.col('Response') == 'Somewhat Disagree')
                                         .then(2)
                                         .when(pl.col('Response') == 'Neither Agree nor Disagree')
                                         .then(3)
                                         .when(pl.col('Response') == 'Somewhat Agree')
                                         .then(4)
                                         .when(pl.col('Response') == 'Strongly Agree')
                                         .then(5)
                    )
       )
Q3_c.head()

Participant,Question,Response,NeedsReverse,TempRecodeValue
u32,str,str,str,i32
1,"""F1""","""Somewhat Agree""","""No""",4
2,"""F1""","""Somewhat Agree""","""No""",4
3,"""F1""","""Strongly Agree""","""No""",5
4,"""F1""","""Somewhat Agree""","""No""",4
5,"""F1""","""Strongly Agree""","""No""",5


In [60]:
# Question 3-d

Q3_d = (Q3_c
       .with_columns(TempReverseValue = pl.when(pl.col('Response') == 'Strongly Disagree')
                                         .then(5)
                                         .when(pl.col('Response') == 'Somewhat Disagree')
                                         .then(4)
                                         .when(pl.col('Response') == 'Neither Agree nor Disagree')
                                         .then(3)
                                         .when(pl.col('Response') == 'Somewhat Agree')
                                         .then(2)
                                         .when(pl.col('Response') == 'Strongly Agree')
                                         .then(1)
                    )
       )
Q3_d.head()

Participant,Question,Response,NeedsReverse,TempRecodeValue,TempReverseValue
u32,str,str,str,i32,i32
1,"""F1""","""Somewhat Agree""","""No""",4,2
2,"""F1""","""Somewhat Agree""","""No""",4,2
3,"""F1""","""Strongly Agree""","""No""",5,1
4,"""F1""","""Somewhat Agree""","""No""",4,2
5,"""F1""","""Strongly Agree""","""No""",5,1


In [61]:
# Question 3-e

Q3_e = (Q3_d
       .with_columns(RecodedValue = pl.when(pl.col('NeedsReverse') == 'No')
                                      .then(pl.col('TempRecodeValue'))
                                      .when(pl.col('NeedsReverse') == 'Yes')
                                      .then(pl.col('TempReverseValue'))
                    )
       )
Q3_e.head()

Participant,Question,Response,NeedsReverse,TempRecodeValue,TempReverseValue,RecodedValue
u32,str,str,str,i32,i32,i32
1,"""F1""","""Somewhat Agree""","""No""",4,2,4
2,"""F1""","""Somewhat Agree""","""No""",4,2,4
3,"""F1""","""Strongly Agree""","""No""",5,1,5
4,"""F1""","""Somewhat Agree""","""No""",4,2,4
5,"""F1""","""Strongly Agree""","""No""",5,1,5


In [62]:
# Question 3-f

Q3_f = (Q3_e
        .with_columns(QuestionType = pl.col('Question').str.split('.').list.get(0))
        #.select(pl.col('QuestionType')).unique()
       )
Q3_f.head()

Participant,Question,Response,NeedsReverse,TempRecodeValue,TempReverseValue,RecodedValue,QuestionType
u32,str,str,str,i32,i32,i32,str
1,"""F1""","""Somewhat Agree""","""No""",4,2,4,"""F1"""
2,"""F1""","""Somewhat Agree""","""No""",4,2,4,"""F1"""
3,"""F1""","""Strongly Agree""","""No""",5,1,5,"""F1"""
4,"""F1""","""Somewhat Agree""","""No""",4,2,4,"""F1"""
5,"""F1""","""Strongly Agree""","""No""",5,1,5,"""F1"""


In [64]:
# Question 3-g

Q3_g = (Q3_f
       .group_by(['Participant','QuestionType'])
       .agg(pl.col('RecodedValue').sum())
       .sort(['Participant','QuestionType'])
       )
Q3_g.head()

Participant,QuestionType,RecodedValue
u32,str,i32
1,"""F1""",31
1,"""F2""",48
1,"""F3""",20
1,"""F4""",17
1,"""F5""",28


In [75]:
# Question 4

Q4 = (Q3_g
     .pivot('QuestionType',
            index = 'Participant',
            values = 'RecodedValue'
           )
     )
Q4.head()

Participant,F1,F2,F3,F4,F5,F6
u32,i32,i32,i32,i32,i32,i32
1,31,48,20,17,28,18
2,31,47,19,17,27,20
3,36,46,19,18,32,17
4,32,54,12,15,30,16
5,37,47,22,19,36,19


In [76]:
FinalTable_lab_1_2 = Q4
FinalTable_lab_1_2.write_csv('./data/FinalTable_lab_1_2.csv')